# 💡 Intro (The Why)

Mental health is a deeply personal and pressing issue, and while AI can't replace a therapist, it *can* offer gentle support, encouragement, and structure for people navigating hard days. 

As part of the GenAI Intensive Capstone with Google, I built **Eve.ai** — a mental wellness companion powered by Gemini models. It helps users reflect, track their mood, and receive supportive affirmations and mindfulness tips — all in a non-judgmental, kind voice.

# 🧱 The Problem

Many people struggle to articulate how they’re feeling, especially in stressful moments. Having a tool that can gently classify emotions, offer a supportive message, and suggest a grounding technique can be empowering.

I wanted to create something that made emotional self-reflection easier — using GenAI.



# 🛠️ How It Works
1. **User inputs a journal-like message**:  
   Example: *“I’m feeling overwhelmed and disconnected from everything lately.”*

2. **Gemini classifies their mood** using few-shot prompts + structured JSON output.  
   Output: `{ "mood": "disconnected", "sentiement_score": -0.87, "suggested_action" : go for walk... }`

3. **Gemini generates a supportive message** based on tone & mood.  
   Example: *“It’s okay to feel disconnected. Just naming your feelings is an act of self-kindness.”*

4. **RAG system fetches a wellness tip** from a preloaded library of CBT/mindfulness techniques using **Gemini embeddings** + **ChromaDB**.

5. The response is displayed with the user’s mood, the affirmation, and the tip.


# 🧪 Sample Output
🧠 Mood: Anxious

💬 Support: You're doing the best you can, and that’s enough. It's okay to pause and breathe.

🌿 Tip: Try the “5-4-3-2-1” grounding technique: list 5 things you see, 4 you can touch, 3 you hear...


# 🤔 Limitations
- Not a replacement for therapy
- Mood classification may oversimplify emotional nuance
- Needs careful prompting to avoid toxic positivity or inappropriate advice


# 🚀 What’s Next
- Track mood over time and visualize progress
- Add journaling insights via LLM-based summaries
- Let users download emotional “reflections” for therapy or self-work


# 🧘 Closing Note
AI can’t heal us — but it can hold space.
That’s what Eve.ai tries to do.

In [1]:
!pip uninstall -qqy jupyterlab kfp  # Remove unused conflicting packages
!pip install -qU "google-genai==1.7.0" "chromadb==0.6.3"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 2.6 MB/s eta 0:

In [2]:
from google import genai
from google.genai import types
from IPython.display import Markdown
import typing_extensions as typ
genai.__version__

'1.7.0'

In [3]:
from google.api_core import retry


is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

genai.models.Models.generate_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.generate_content)

In [4]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

client = genai.Client(api_key=GOOGLE_API_KEY)

In [5]:
import typing_extensions as typing
import json

class classifyMoods(typing.TypedDict):
    mood : str
    sentiement_score : float
    suggested_action : str
    
def analyzeMood(entryText):
    few_shot_prompt = (
            "You are a mental health assistant powered by Google’s Gemini model. "
            "Here are some examples:\n\n"
            "Example 1:\n"
            "Entry: 'I'm feeling overwhelmed with work and stress.'\n"
            "Output: {\"mood\": \"stressed\", \"sentiment_score\": -0.8, \"suggested_action\": \"try a short breathing exercise\"}\n\n"
            "Example 2:\n"
            "Entry: 'I woke up feeling joyful and grateful today.'\n"
            "Output: {\"mood\": \"positive\", \"sentiment_score\": 0.7, \"suggested_action\": \"write a gratitude note\"}\n\n"
            "Now, classify the following entry:\n"
             "Entry: \"{entry_text}\"\n"
            "Output:"
        )
    

    reply = client.models.generate_content(
        model = 'gemini-2.0-flash',
        config = types.GenerateContentConfig(
            temperature = 1.0,
            top_p = 0.69,
            response_mime_type="application/json",
            response_schema=classifyMoods,
        ),
        contents = [entryText,few_shot_prompt],
    )

    return reply

text = "I’m feeling overwhelmed and disconnected from everything lately."
#try changing it into "im feeling anxious"
user_mood = analyzeMood(text).text.strip()
moods_data = json.loads(user_mood)

Markdown(user_mood)

{
  "mood": "disconnected",
  "sentiement_score": -0.6,
  "suggested_action": "try mindfulness meditation"
}

In [6]:
def generate_supportive_response(entry_text):
    prompt = f"""You are a caring mental health companion. Provide a supportive, empathetic response 
    to this journal entry, offering a brief mindfulness or CBT-based suggestion.
    EXAMPLES = {
  {
    "input": "I'm feeling overwhelmed with work.",
    "response": "That sounds really tough. It's okay to feel overwhelmed sometimes. Do you want to talk more about what's stressing you out?"
  },
  {
    "input": "I'm doing okay, just tired.",
    "response": "Thanks for sharing. It's okay to have low energy days. Maybe a little rest and kindness toward yourself would help."
  },
  {
    "input": "I can't focus and I feel anxious.",
    "response": "That sounds like you're under a lot of pressure. Let's take a deep breath together. You're not alone in this."
  }
  }
    Journal Entry: "{entry_text}".
    """
    response = client.models.generate_content(
        model='gemini-2.0-flash',
        config = types.GenerateContentConfig(
            max_output_tokens=100,
            top_p = 0.5,
            temperature = 1
        ),
        contents=prompt
    )
    
    return response

query = generate_supportive_response(text)
Markdown(query.text.strip())


I hear you. Feeling overwhelmed and disconnected can be really draining. It's completely understandable that you're feeling this way. Would you like to explore what might be contributing to these feelings? Sometimes, when we feel disconnected, it helps to gently reconnect with our senses. Perhaps try a quick mindfulness exercise: Take a moment to notice five things you can see, four things you can touch, three things you can hear, two things you can smell, and one thing you can taste. This can

In [7]:
DOCUMENT1 = """Breathing Exercise for Anxiety Relief  
When you're feeling anxious or overwhelmed, a simple breathing exercise can help calm your mind. 
Try this: Inhale slowly for 4 seconds, hold your breath for 4 seconds, exhale slowly for 4 seconds, then hold again for 4 seconds. 
Repeat this cycle a few times. This technique helps regulate your nervous system and promotes a sense of control."""

DOCUMENT2 = """Practicing Gratitude  
Gratitude can improve your overall mood and shift your focus away from negative thoughts. 
Each morning or evening, try writing down 3 things you're grateful for — big or small. 
Over time, this simple habit can boost happiness and reduce stress."""

DOCUMENT3 = """Taking Breaks When You're Mentally Drained  
It's okay to pause when your brain feels foggy or tired. 
Step away from what you're doing and give yourself a 5–10 minute break. 
Stretch, drink water, or go for a short walk. These small resets can improve focus and reduce burnout over time."""

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

import pandas as pd

# Load your actual wellness tips dataset
df = pd.read_json("/kaggle/input/wellness-tips/wellness_tips.json")

# Convert rows into list of string tips (document style)
document = [row["tip"] for _, row in df.iterrows()]
print("sucessfully loaded")

sucessfully loaded


In [8]:
from chromadb import Documents, EmbeddingFunction, Embeddings
from google.api_core import retry

from google.genai import types


# Define a helper to retry when per-minute quota is reached.
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})


class GeminiEmbeddingFunction(EmbeddingFunction):
    # Specify whether to generate embeddings for documents, or queries
    document_mode = True

    @retry.Retry(predicate=is_retriable)
    def __call__(self, input: Documents) -> Embeddings:
        if self.document_mode:
            embedding_task = "retrieval_document"
        else:
            embedding_task = "retrieval_query"

        response = client.models.embed_content(
            model="models/text-embedding-004",
            contents=input,
            config=types.EmbedContentConfig(
                task_type=embedding_task,
            ),
        )
        return [e.values for e in response.embeddings]
        
print("Success")

Success


In [9]:
# Adding Data to database
import chromadb

DB_NAME = "googlecardb"

embed_fn = GeminiEmbeddingFunction()
embed_fn.document_mode = True

chroma_client = chromadb.Client()
db = chroma_client.get_or_create_collection(name=DB_NAME, embedding_function=embed_fn)

db.add(documents=documents, ids=[str(i) for i in range(len(documents))])

In [10]:
# Switch to query mode when generating embeddings.
embed_fn.document_mode = False

# Search the Chroma DB using the specified query.
mood = moods_data["mood"]
query = f"I'm feeling { mood }, what can I do?"

result = db.query(query_texts=[query], n_results=1)
[all_passages] = result["documents"]

Markdown(all_passages[0])

Taking Breaks When You're Mentally Drained  
It's okay to pause when your brain feels foggy or tired. 
Step away from what you're doing and give yourself a 5–10 minute break. 
Stretch, drink water, or go for a short walk. These small resets can improve focus and reduce burnout over time.

In [11]:
query_oneline = query.replace("\n", " ")

# This prompt is where you can specify any guidance on tone, or what topics the model should stick to, or avoid.
prompt = f"""You are a compassionate and friendly mental health assistant. 
Use the reference passage below to suggest helpful actions based on the user's mood. 
Respond in a supportive and conversational tone, avoiding technical jargon. 
Offer clear, relatable advice that's easy to follow. 
If the passage doesn't help answer the question, it's okay to skip it.

QUESTION: {query_oneline}
"""


# Add the retrieved documents to the prompt.
for passage in all_passages:
    passage_oneline = passage.replace("\n", " ")
    prompt += f"PASSAGE: {passage_oneline}\n"

print(prompt)

You are a compassionate and friendly mental health assistant. 
Use the reference passage below to suggest helpful actions based on the user's mood. 
Respond in a supportive and conversational tone, avoiding technical jargon. 
Offer clear, relatable advice that's easy to follow. 
If the passage doesn't help answer the question, it's okay to skip it.

QUESTION: I'm feeling disconnected, what can I do?
PASSAGE: Taking Breaks When You're Mentally Drained   It's okay to pause when your brain feels foggy or tired.  Step away from what you're doing and give yourself a 5–10 minute break.  Stretch, drink water, or go for a short walk. These small resets can improve focus and reduce burnout over time.



In [12]:
answer = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=prompt)

Markdown(answer.text)

Hey, I hear you. Feeling disconnected can be a real drag. It sounds like you need a little reset. The good news is, even small things can help!

How about trying a quick break? Seriously, even just 5-10 minutes can make a difference. Maybe step away from whatever you're doing, stretch your arms above your head, and take a few deep breaths. Grabbing a glass of water or taking a short walk could also help you feel a bit more grounded. Sometimes, just a change of scenery is all it takes to shake off that disconnected feeling. Let me know if you want to explore other options too!
